<table>
<tr>
<td width=15%><img src="https://www.nitrkl.ac.in/assets/images/logo.png"></img></td>
<td><center><h1>National Institute of Technology
Rourkela, Odisha, India, 769008</h1><h2>Department of Computer Science Engineering</h2></center></td>
</tr>
<tr><td colspan=2><center><h3>National Workshop on CFTSFDL-2024</h3></br>
            <b>(Day-2 Hands-on)</b></center></td></tr>
</table>

In [ ]:
# Import required packages
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Flatten,LSTM, GRU,Bidirectional,TimeDistributed
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.metrics import RootMeanSquaredError,mean_absolute_percentage_error
from keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dropout

In [ ]:
# split a univariate time series into patterns
def get_Patterns(TSeries, n_inputs,h):
    X,y,z = pd.DataFrame(np.zeros((len(TSeries)-n_inputs-h+1,n_inputs))), pd.DataFrame(), pd.DataFrame()
    for i in range(len(TSeries)):
        # find the end of this pattern
        end_ix = i + n_inputs + h - 1
        # check if we are beyond the time series
        if end_ix > len(TSeries)-1:
            break
        # gather input and output parts of the pattern
        for j in range(n_inputs):
            X.loc[i,j]=TSeries.iloc[i+j,0]
        i=i+n_inputs
        #y=y.append(TSeries.iloc[end_ix], ignore_index = True)
        y=pd.concat([y, TSeries.iloc[end_ix]], ignore_index=True)
    return pd.DataFrame(X),pd.DataFrame(y)

In [ ]:
# originalData should be a Column Vectored DataFrame
def minmaxNorm(originalData, lenTrainValidation):
    # Maximum Value
    max2norm=max(originalData.iloc[0:lenTrainValidation,0])
    # Minimum Value
    min2norm=min(originalData.iloc[0:lenTrainValidation,0])
    lenOriginal=len(originalData)
    normalizedData=np.zeros(lenOriginal)   
    normalizedData = []
    #Normalize using Min-Max Normalization
    for i in range (lenOriginal):
        normalizedData.append((originalData.iloc[i]-min2norm)/(max2norm-min2norm))    
    return pd.DataFrame(normalizedData)

In [ ]:
# originalData and forecastedData should be Column Vectored DataFrames
def minmaxDeNorm( originalData, forecastedData, lenTrainValidation):
    # Maximum Value
    max2norm=max(originalData.iloc[0:lenTrainValidation,0])
    # Minimum Value
    min2norm=min(originalData.iloc[0:lenTrainValidation,0])
    lenOriginal=len(originalData)
    denormalizedData=[]   
    #De-Normalize using Min-Max Normalization
    for i in range (lenOriginal):
        denormalizedData.append((forecastedData.iloc[i]*(max2norm-min2norm))+min2norm)  
    return pd.DataFrame(denormalizedData)

In [ ]:
# Timeseries_Data and forecasted_value should be Column Vectored DataFrames
def findRMSE( Timeseries_Data, forecasted_value,lenTrainValidation):
    l=Timeseries_Data.shape[0]
    lenTest=l-lenTrainValidation
    trainRMSE=0;
    for i in range (lenTrainValidation):
        trainRMSE=trainRMSE+np.power((forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0]),2) 
    trainRMSE=np.sqrt(trainRMSE/lenTrainValidation)

    testRMSE=0;
    for i in range (lenTrainValidation,l,1):
        testRMSE=testRMSE+np.power((forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0]),2)
    testRMSE=np.sqrt(testRMSE/lenTest)
    return trainRMSE, testRMSE 

In [ ]:
# Timeseries_Data and forecasted_value should be Column Vectored DataFrames
def findMAE( Timeseries_Data, forecasted_value,lenTrainValidation):
    l=Timeseries_Data.shape[0]
    lenTest=l-lenTrainValidation
    trainMAE=0;
    for i in range (lenTrainValidation):
        trainMAE=trainMAE+np.abs(forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0]) 
    trainMAE=(trainMAE/(lenTrainValidation));

    testMAE=0;
    for i in range (lenTrainValidation,l,1):
        testMAE=testMAE+np.abs(forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0])
    testMAE=(testMAE/lenTest);
    return trainMAE, testMAE

In [ ]:
def Find_Fitness_LSTM(x,y,lenValid,lenTest):
    NOP=len(y)
    n_features=1
    lenTrain=NOP-lenValid-lenTest
    xTrain=x.iloc[0:lenTrain,:]
    xValid=x.iloc[lenTrain:(lenTrain+lenValid),:]
    xTest=x.iloc[(lenTrain+lenValid):NOP,:]
    yTrain=y.iloc[0:lenTrain,0]
    yValid=y.iloc[lenTrain:(lenTrain+lenValid),0]
    yTest=y.iloc[(lenTrain+lenValid):NOP,0]
    model1 = Sequential()
    model1.add(LSTM(256, activation='relu', input_shape=(LagLength,n_features)))
    model1.add(Flatten())
    model1.add(Dense(128, activation='relu'))
    model1.add(Dense(64, activation='relu'))
    model1.add(Dense(32, activation='relu'))
    model1.add(Dense(1,activation='linear'))
    checkpoint_filepath = '/cp.keras'
    cp1=ModelCheckpoint(checkpoint_filepath,save_best_only='True')
    model1.compile(loss=MeanSquaredError(),optimizer=Adam(learning_rate=0.01),metrics=[RootMeanSquaredError(),mean_absolute_percentage_error])
    # fit
    model1.fit(xTrain, yTrain,validation_data=(xValid,yValid), epochs=50, verbose=1)
    yhatNorm=model1.predict(x).flatten().reshape(x.shape[0],1)
    return pd.DataFrame(yhatNorm)

In [ ]:
#Read the AQI Time Series
from google.colab import files
uploaded = files.upload()

In [ ]:
#Read the Time Series Dataset
Timeseries_Data=pd.read_csv('AQI.csv',header=None)
LagLength=24
h=1
lt=Timeseries_Data.shape[0]
lenTrain=int(round(lt*0.7))
lenValidation=int(round(lt*0.15))
lenTest=int(lt-lenTrain-lenValidation)
# NORMALIZE THE DATA
normalizedData=minmaxNorm(Timeseries_Data,lenTrain+lenValidation);
# Transform the Time Series into Patterns Using Sliding Window
X, y = get_Patterns(normalizedData, LagLength, h)
name='LSTM'
file1='./'+str(name)+"_Accuracy.xlsx"
file2='./'+str(name)+"_Forecasts.xlsx"
Forecasts=pd.DataFrame()
Accuracy=pd.DataFrame()

ynorm1=Find_Fitness_LSTM(X,y,lenValidation,lenTest) 
ynorm=pd.DataFrame(normalizedData.iloc[0:(LagLength+h-1),0])
ynorm=pd.concat([ynorm, ynorm1], ignore_index=True)
yhat=minmaxDeNorm(Timeseries_Data, ynorm, lenTrain+lenValidation)
Accuracy.loc[0,0],Accuracy.loc[0,1]=findRMSE( Timeseries_Data,yhat,lenTrain+lenValidation)
Accuracy.loc[0,2],Accuracy.loc[0,3]=findMAE( Timeseries_Data,yhat,lenTrain+lenValidation)
Forecasts=pd.concat([Forecasts, yhat.T], ignore_index=True)
Accuracy.to_excel(file1,sheet_name='Accuracy',index=False)
Forecasts.to_excel(file2,sheet_name='Forecasts',index=False)
print(Accuracy)

11/11 [==============================] - 2s 163ms/step - loss: 3.2477e-04 - root_mean_squared_error: 0.0180 - mean_absolute_percentage_error: 45782.0977 - val_loss: 5.5485e-04 - val_root_mean_squared_error: 0.0236 - val_mean_absolute_percentage_error: 3.8501
Epoch 22/50
11/11 [==============================] - 1s 135ms/step - loss: 4.2973e-04 - root_mean_squared_error: 0.0207 - mean_absolute_percentage_error: 24325.6738 - val_loss: 4.6673e-04 - val_root_mean_squared_error: 0.0216 - val_mean_absolute_percentage_error: 2.8881
Epoch 23/50
11/11 [==============================] - 2s 138ms/step - loss: 5.4685e-04 - root_mean_squared_error: 0.0234 - mean_absolute_percentage_error: 100013.6172 - val_loss: 5.6858e-04 - val_root_mean_squared_error: 0.0238 - val_mean_absolute_percentage_error: 3.6687
Epoch 24/50
11/11 [==============================] - 2s 141ms/step - loss: 4.1403e-04 - root_mean_squared_error: 0.0203 - mean_absolute_percentage_error: 53973.3789 - val_loss: 3.5775e-04 - val_root